In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 98.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.9 MB/s eta 0:00:00


In [2]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

In [3]:
# Load the pretrained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [4]:
# Set device (CPU or GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Tokenizer에게 Text 전달하고 예측값 retur

In [19]:
# Text classification function
def classify_text(text):
    # Tokenize input text
    # tolnizer에게 text 전달
    inputs = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        padding='max_length',
        truncation=True,
        max_length=128, ## 문장길이가 128이 아닐 경우 padding 추가
        return_tensors='pt'
    )
    print(inputs)
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    # Forward pass through the model
    with torch.no_grad():
        outputs = model.to('cuda')(input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=1).squeeze(dim=0)
    # logits값과 softmax를 통한 각기 확률 얻기
    predicted_class = torch.argmax(probabilities).item()

    return predicted_class, probabilities

In [20]:
# Example usage
text_to_classify = "This is an example sentence."
predicted_class, probabilities = classify_text(text_to_classify)

{'input_ids': tensor([[ 101, 2023, 2003, 2019, 2742, 6251, 1012,  102,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0,

## Bert-base-uncased 에서는 2개의 클래스에 대해서 훈련되었기 때문에 2개의 예측이 나타남

In [7]:
# Print the predicted class and probabilities
print(f"Predicted class: {predicted_class}")
print("Probabilities:")
for i, prob in enumerate(probabilities):
    print(f"Class {i}: {prob.item()}")

Predicted class: 0
Probabilities:
Class 0: 0.5780633687973022
Class 1: 0.42193660140037537


In [8]:
import torch
from transformers import BertTokenizer, BertForMaskedLM

In [9]:
# Load the pretrained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForMaskedLM.from_pretrained(model_name)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
# Set device (CPU or GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [23]:
# Next word prediction function
def predict_next_word(text):
    # Tokenize input text
    tokenized_text = tokenizer.tokenize(text)
    print("tokenized_text : ",tokenized_text)
    masked_index = tokenized_text.index('[MASK]')
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    print(f'index _token : {indexed_tokens}')
    # Convert tokens to tensor
    tokens_tensor = torch.tensor([indexed_tokens]).to(device)

    # Forward pass through the model
    with torch.no_grad():
        outputs = model.to('cuda')(tokens_tensor)
    print(outputs)
    ## 예측에 대한 것은 token id로 제공됌
    predictions = outputs[0][0, masked_index].topk(k=5).indices.tolist()
    print(f'predictions : {predictions}')
    # 예측된 ids값들을 token으로 변경

    predicted_tokens = []
    for token_index in predictions:
        predicted_token = tokenizer.convert_ids_to_tokens([token_index])[0]
        predicted_tokens.append(predicted_token)

    return predicted_tokens

In [24]:
# Example usage
text_with_mask = "I want to [MASK] a pizza for dinner."
predicted_tokens = predict_next_word(text_with_mask)

# Print the predicted tokens
print(predicted_tokens)

tokenized_text :  ['i', 'want', 'to', '[MASK]', 'a', 'pizza', 'for', 'dinner', '.']
index _token : [1045, 2215, 2000, 103, 1037, 10733, 2005, 4596, 1012]
MaskedLMOutput(loss=None, logits=tensor([[[ -8.0052,  -7.9416,  -7.9914,  ...,  -7.7750,  -7.5297,  -5.0479],
         [ -7.8714,  -7.5587,  -7.7266,  ...,  -6.9916,  -6.7327,  -5.6759],
         [ -9.3953,  -8.8095,  -9.1847,  ...,  -7.7748,  -7.6339,  -5.9241],
         ...,
         [ -8.1732,  -7.7539,  -7.9438,  ...,  -7.0557,  -6.6260,  -5.1185],
         [ -9.4237,  -9.2842,  -9.3143,  ...,  -8.0756,  -7.8228,  -8.0178],
         [-10.1565,  -9.9998, -10.0675,  ...,  -9.7280,  -8.8740,  -7.8272]]],
       device='cuda:0'), hidden_states=None, attentions=None)
predictions : [2031, 2131, 2022, 2191, 2202]
['have', 'get', 'be', 'make', 'take']
